In [1]:
%matplotlib inline
import glob
import pandas as pd
import mne
import matplotlib.pyplot as plt
from io import StringIO
import mne
from mne.io import read_raw_eeglab, read_epochs_eeglab
import numpy as np
from scipy import signal
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm_notebook
from sklearn.svm import SVR

import autosklearn.regression
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import multiprocessing
from tpot import TPOTRegressor
from oct2py import octave

from joblib import Parallel, delayed
import multiprocessing
from joblib import wrap_non_picklable_objects
import json
import pickle
import os.path
from mpl_toolkits.mplot3d import axes3d
from timeit import default_timer as timer
from datetime import timedelta

/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/pyparsing.py:2927: FutureWarning: Possible set intersection at position 3
  self.re = re.compile( self.reString )
/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
eeglab_path = '/home/raquib/Documents/MATLAB/eeglab2019_0/functions/'
octave.addpath(eeglab_path + 'guifunc');
octave.addpath(eeglab_path + 'popfunc');
octave.addpath(eeglab_path + 'adminfunc');
octave.addpath(eeglab_path + 'sigprocfunc');
octave.addpath(eeglab_path + 'miscfunc');

In [3]:
experiment = 'data/original/*/*'
meps = sorted(glob.glob(experiment + '/mep/*/*.txt'))
mep_present = len(meps) > 0
eegs = sorted(glob.glob(experiment + '/eeg/*/clean-prestimulus.set'))
eeg_present = len(eegs) > 0
cmaps = sorted(glob.glob(experiment + '/cmap/*.xlsx'))
cmap_present = len(cmaps) > 0
all_present = mep_present and eeg_present and cmap_present
print(all_present)

True


In [4]:
print('EEG count: ' + str(len(eegs)))
print('MEP count: ' + str(len(meps)))
print('CMAP count: ' + str(len(cmaps)))

EEG count: 44
MEP count: 77
CMAP count: 22


In [5]:
eegs

['data/original/sub03/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
 'data/original/sub03/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
 'data/original/sub03/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
 'data/original/sub03/exp02/eeg/SP 110RMT r1/clean-prestimulus.set',
 'data/original/sub03/exp02/eeg/SP 110RMT r2/clean-prestimulus.set',
 'data/original/sub03/exp03/eeg/SP 110RMT r1/clean-prestimulus.set',
 'data/original/sub03/exp03/eeg/SP 110RMT r2/clean-prestimulus.set',
 'data/original/sub03/exp03/eeg/SP 110RMT r3/clean-prestimulus.set',
 'data/original/sub04/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
 'data/original/sub04/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
 'data/original/sub04/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
 'data/original/sub05/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
 'data/original/sub05/exp02/eeg/SP 120RMT r2/clean-prestimulus.set',
 'data/original/sub06/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
 'data/original/sub06/exp01/eeg/SP

In [6]:
eegs = [
    'data/original/sub03/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub03/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub03/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
#     'data/original/sub03/exp02/eeg/SP 110RMT r1/clean-prestimulus.set', NO CMAP
#     'data/original/sub03/exp02/eeg/SP 110RMT r2/clean-prestimulus.set', NO CMAP
#     'data/original/sub03/exp03/eeg/SP 110RMT r1/clean-prestimulus.set', NO CMAP
    'data/original/sub03/exp03/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub03/exp03/eeg/SP 110RMT r3/clean-prestimulus.set',
#     'data/original/sub04/exp01/eeg/SP 110RMT r1/clean-prestimulus.set', NO CMAP
#     'data/original/sub04/exp01/eeg/SP 110RMT r2/clean-prestimulus.set', NO CMAP
#     'data/original/sub04/exp01/eeg/SP 110RMT r3/clean-prestimulus.set', NO CMAP
#     'data/original/sub05/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
    'data/original/sub06/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub06/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub06/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
    'data/original/sub06/exp02/eeg/SP 110RMT/clean-prestimulus.set',
    'data/original/sub07/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub07/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub07/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
    'data/original/sub08/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub08/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub08/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
    'data/original/sub08/exp02/eeg/SP 110RMT/clean-prestimulus.set',
    'data/original/sub08/exp03/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub08/exp03/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub08/exp03/eeg/SP 110RMT r3/clean-prestimulus.set',
    'data/original/sub12/exp02/eeg/SP 110RMT/clean-prestimulus.set',
#     'data/original/sub13/exp01/eeg/SP 110RMT/clean-prestimulus.set', LEFT HANDED
    'data/original/sub14/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub15/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub15/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub15/exp01/eeg/SP 110RMT r3/clean-prestimulus.set',
    'data/original/sub16/exp01/eeg/SP 110RMT r1/clean-prestimulus.set',
    'data/original/sub16/exp01/eeg/SP 110RMT r2/clean-prestimulus.set',
    'data/original/sub16/exp01/eeg/SP 110RMT r3/clean-prestimulus.set'
]

# Read and process EEG

In [7]:
def read_eeg(path):
    eeg = octave.pop_loadset(path)
    new_trial_list = []
    for i in range(eeg.data.shape[2]):
        trial = eeg.data[:, :, i]
        time = np.linspace(-1000, -20, num=trial.shape[1])
        trial = pd.DataFrame(np.transpose(trial), columns=eeg.chanlocs.labels[0])
        trial['time'] = time
        new_trial_list.append(trial)
    return new_trial_list

def crop_trials(trial_list, duration_millis=500, sampling_rate=2048):
    new_trial_list = []
    for trial in trial_list:
        samples_to_pick = duration_millis * sampling_rate / 1000
        new_trial_list.append(trial.tail(int(samples_to_pick)))
    return new_trial_list, samples_to_pick

# Calculate EEG area.
def calculate_eeg_area(epoch_df, sf=2048):
    y = epoch_df.drop('time', axis=1).mean(axis=1)
    b2, a2 = signal.butter(4, 200/(sf/2), btype='lowpass')
    envelope = signal.filtfilt(b2, a2, np.abs(y))
    area = np.trapz(envelope, epoch_df['time'].values)
    return area

# Calculate EEG frequency.
def calculate_eeg_frequency(channel):
    sf = 2048
    win = 4 * sf
    freqs, psd = signal.welch(channel, sf, nperseg=win)
    return freqs, psd

def calculate_eeg_max_amplitude(epoch_df):
    avg = epoch_df.mean(axis=1)
    return np.max(avg.values)

def band_max(freq, psd, interval):
    indices = []
    for el in freq:
        indices.append(el in interval)
    freq = freq[indices]
    psd = psd[indices]
    if (len(psd) == 0):
        return 0, 0
    i = np.argmax(np.abs(psd))
    return freq[i], psd[i]

def filter_electrodes(trial, which='all'):
    time_column = trial['time']
    if which == 'ltm1':
        channel_names = ['FC5','FC1','C3','CP5','CP1','FC3','C5','C1','CP3']
    elif which == 'rtm1':
        channel_names = ['FC6','FC2','C4','CP6','CP2','FC4','C6','C2','CP4']
    elif which == 'central':
        channel_names = ['Fz','FCz','Cz','F1','FC1','C1','C2','FC2','F2']
    else:
        channel_names = ['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6', 'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'O2', 'EOG', 'AF7', 'AF3', 'AF4', 'AF8', 'F5', 'F1', 'F2', 'F6', 'FC3', 'FCz', 'FC4', 'C5', 'C1', 'C2', 'C6', 'CP3', 'CP4', 'P5', 'P1', 'P2', 'P6', 'PO5', 'PO3', 'PO4', 'PO6', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'Oz']
    trial = trial[channel_names]
    trial['time'] = time_column
    return trial

def read_wavelets(sub, exp, run, epoch_num):
    path = 'wavelets-69/' + sub + '-' + exp + '-' + run + '-' + str(epoch_num)
    with open(path + '-central.pickle', 'rb') as f:
        central = pickle.load(f)
    with open(path + '-ltm1.pickle', 'rb') as f:
        ltm1 = pickle.load(f)
    with open(path + '-rtm1.pickle', 'rb') as f:
        rtm1 = pickle.load(f)
    with open(path + '-all.pickle', 'rb') as f:
        all_channels = pickle.load(f)
    with open(path + '-ltdlpfc.pickle', 'rb') as f:
        ltdlpfc = pickle.load(f)
    with open(path + '-rtdlpfc.pickle', 'rb') as f:
        rtdlpfc = pickle.load(f)
    with open(path + '-ltparietal.pickle', 'rb') as f:
        ltparietal = pickle.load(f)
    with open(path + '-rtparietal.pickle', 'rb') as f:
        rtparietal = pickle.load(f)
    with open(path + '-occipital.pickle', 'rb') as f:
        occipital = pickle.load(f)
    with open(path + '-ltoccipital.pickle', 'rb') as f:
        ltoccipital = pickle.load(f)
    with open(path + '-rtoccipital.pickle', 'rb') as f:
        rtoccipital = pickle.load(f)
    return all_channels, ltm1, rtm1, central, ltdlpfc, rtdlpfc, ltparietal, rtparietal, occipital, ltoccipital, rtoccipital

def wavelet_band_max(df, interval):
    indices = []
    for el in (df.index * 1000):
        indices.append(el in interval)
    df = df[indices]
    if (df.shape[0] == 0):
        return 0, 0, 0, 0
    return df.mean(axis=1).max(), df.mean(axis=1).argmax() * 1000, df.mean(axis=0).max(), df.mean(axis=0).argmax()

def normalize_image(image, mx=1, mn=0):
    wt_min = image.min().min()
    std = (image.values - wt_min) / (image.max().max() - wt_min)
    image.loc[:, :] = std * (mx - mn) + mn
    return image

def crop_wavelet(wt, start_time_millis = -25, end_time_millis = -20, start_freq = 2, end_freq = 20):
    wt = wt[(wt.index * 1000 > start_freq) * (wt.index * 1000 < end_freq)]
    wt = wt.loc[:, wt.columns.isin(wt.columns[(wt.columns >= (start_time_millis/1000)) * (wt.columns <= (end_time_millis/1000))])]
    wt.columns = np.round(wt.columns, 3)
    wt.index = np.round(wt.index*1000, 2)
    wt = normalize_image(wt)
    return wt

def populate_corr(wt, df_corr_el):
    for time, column in wt.iteritems():
        for freq, row in pd.DataFrame(column).iterrows():
            df_corr_el['power_' + str(time) + '_' + str(freq)] = row.values[0]
    return df_corr_el

# Read features file

In [8]:
features_filename = '55-features-v2.xlsx'

In [9]:
df = pd.read_excel(features_filename, index_col=0)

p1 = np.percentile(df['mep_category_cmap'], 50)
cat = np.ones(len(df['mep_category_cmap'])) * (df['mep_category_cmap'] > p1)
df['mep_category_cmap_across_subjects'] = cat

# Prepare for anova and t-test

In [10]:
def get_dict_of_avg_power(epoch):
    wt_all, wt_ltm1, wt_rtm1, wt_central, wt_ltdlpfc, wt_rtdlpfc, wt_ltparietal, wt_rtparietal, wt_occipital, wt_ltoccipital, wt_rtoccipital = read_wavelets(epoch['sub'], epoch['exp'], epoch['run'], epoch['epoch'])
    wt_all_alpha_mean = crop_wavelet(wt_all, start_time_millis=-420, end_time_millis=-20, start_freq=8, end_freq=13).mean().mean()
    wt_all_beta_mean = crop_wavelet(wt_all, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=30).mean().mean()
    wt_all_beta_gamma_mean = crop_wavelet(wt_all, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=80).mean().mean()
    dct =  {
        'wt_all_alpha_mean': crop_wavelet(wt_all, start_time_millis=-420, end_time_millis=-20, start_freq=8, end_freq=13).values.flatten().mean(),
        'wt_all_beta_mean': crop_wavelet(wt_all, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=30).values.flatten().mean(),
        'wt_all_beta_gamma_mean': crop_wavelet(wt_all, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=80).values.flatten().mean(),
        'wt_all_gamma_mean': crop_wavelet(wt_all, start_time_millis=-420, end_time_millis=-20, start_freq=30, end_freq=80).values.flatten().mean(),
        
        'wt_ltm1_alpha_mean': crop_wavelet(wt_ltm1, start_time_millis=-420, end_time_millis=-20, start_freq=8, end_freq=13).values.flatten().mean(),
        'wt_ltm1_beta_mean': crop_wavelet(wt_ltm1, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=30).values.flatten().mean(),
        'wt_ltm1_beta_gamma_mean': crop_wavelet(wt_ltm1, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=80).values.flatten().mean(),
        'wt_ltm1_gamma_mean': crop_wavelet(wt_ltm1, start_time_millis=-420, end_time_millis=-20, start_freq=30, end_freq=80).values.flatten().mean(),
        
        'wt_rtm1_alpha_mean': crop_wavelet(wt_rtm1, start_time_millis=-420, end_time_millis=-20, start_freq=8, end_freq=13).values.flatten().mean(),
        'wt_rtm1_beta_mean': crop_wavelet(wt_rtm1, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=30).values.flatten().mean(),
        'wt_rtm1_beta_gamma_mean': crop_wavelet(wt_rtm1, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=80).values.flatten().mean(),
        'wt_rtm1_gamma_mean': crop_wavelet(wt_rtm1, start_time_millis=-420, end_time_millis=-20, start_freq=30, end_freq=80).values.flatten().mean(),
        
        'wt_central_alpha_mean': crop_wavelet(wt_central, start_time_millis=-420, end_time_millis=-20, start_freq=8, end_freq=13).values.flatten().mean(),
        'wt_central_beta_mean': crop_wavelet(wt_central, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=30).values.flatten().mean(),
        'wt_central_beta_gamma_mean': crop_wavelet(wt_central, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=80).values.flatten().mean(),
        'wt_central_gamma_mean': crop_wavelet(wt_central, start_time_millis=-420, end_time_millis=-20, start_freq=30, end_freq=80).values.flatten().mean(),
        
        'wt_ltdlpfc_alpha_mean': crop_wavelet(wt_ltdlpfc, start_time_millis=-420, end_time_millis=-20, start_freq=8, end_freq=13).values.flatten().mean(),
        'wt_ltdlpfc_beta_mean': crop_wavelet(wt_ltdlpfc, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=30).values.flatten().mean(),
        'wt_ltdlpfc_beta_gamma_mean': crop_wavelet(wt_ltdlpfc, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=80).values.flatten().mean(),
        'wt_ltdlpfc_gamma_mean': crop_wavelet(wt_ltdlpfc, start_time_millis=-420, end_time_millis=-20, start_freq=30, end_freq=80).values.flatten().mean(),
        
        'wt_rtdlpfc_alpha_mean': crop_wavelet(wt_rtdlpfc, start_time_millis=-420, end_time_millis=-20, start_freq=8, end_freq=13).values.flatten().mean(),
        'wt_rtdlpfc_beta_mean': crop_wavelet(wt_rtdlpfc, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=30).values.flatten().mean(),
        'wt_rtdlpfc_beta_gamma_mean': crop_wavelet(wt_rtdlpfc, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=80).values.flatten().mean(),
        'wt_rtdlpfc_gamma_mean': crop_wavelet(wt_rtdlpfc, start_time_millis=-420, end_time_millis=-20, start_freq=30, end_freq=80).values.flatten().mean(),
        
        'wt_occipital_alpha_mean': crop_wavelet(wt_occipital, start_time_millis=-420, end_time_millis=-20, start_freq=8, end_freq=13).values.flatten().mean(),
        'wt_occipital_beta_mean': crop_wavelet(wt_occipital, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=30).values.flatten().mean(),
        'wt_occipital_beta_gamma_mean': crop_wavelet(wt_occipital, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=80).values.flatten().mean(),
        'wt_occipital_gamma_mean': crop_wavelet(wt_occipital, start_time_millis=-420, end_time_millis=-20, start_freq=30, end_freq=80).values.flatten().mean(),
        
        'wt_ltoccipital_alpha_mean': crop_wavelet(wt_ltoccipital, start_time_millis=-420, end_time_millis=-20, start_freq=8, end_freq=13).values.flatten().mean(),
        'wt_ltoccipital_beta_mean': crop_wavelet(wt_ltoccipital, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=30).values.flatten().mean(),
        'wt_ltoccipital_beta_gamma_mean': crop_wavelet(wt_ltoccipital, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=80).values.flatten().mean(),
        'wt_ltoccipital_gamma_mean': crop_wavelet(wt_ltoccipital, start_time_millis=-420, end_time_millis=-20, start_freq=30, end_freq=80).values.flatten().mean(),
        
        'wt_rtoccipital_alpha_mean': crop_wavelet(wt_rtoccipital, start_time_millis=-420, end_time_millis=-20, start_freq=8, end_freq=13).values.flatten().mean(),
        'wt_rtoccipital_beta_mean': crop_wavelet(wt_rtoccipital, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=30).values.flatten().mean(),
        'wt_rtoccipital_beta_gamma_mean': crop_wavelet(wt_rtoccipital, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=80).values.flatten().mean(),
        'wt_rtoccipital_gamma_mean': crop_wavelet(wt_rtoccipital, start_time_millis=-420, end_time_millis=-20, start_freq=30, end_freq=80).values.flatten().mean(),
        
        'wt_ltparietal_alpha_mean': crop_wavelet(wt_ltparietal, start_time_millis=-420, end_time_millis=-20, start_freq=8, end_freq=13).values.flatten().mean(),
        'wt_ltparietal_beta_mean': crop_wavelet(wt_ltparietal, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=30).values.flatten().mean(),
        'wt_ltparietal_beta_gamma_mean': crop_wavelet(wt_ltparietal, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=80).values.flatten().mean(),
        'wt_ltparietal_gamma_mean': crop_wavelet(wt_ltparietal, start_time_millis=-420, end_time_millis=-20, start_freq=30, end_freq=80).values.flatten().mean(),
        
        'wt_rtparietal_alpha_mean': crop_wavelet(wt_rtparietal, start_time_millis=-420, end_time_millis=-20, start_freq=8, end_freq=13).values.flatten().mean(),
        'wt_rtparietal_beta_mean': crop_wavelet(wt_rtparietal, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=30).values.flatten().mean(),
        'wt_rtparietal_beta_gamma_mean': crop_wavelet(wt_rtparietal, start_time_millis=-420, end_time_millis=-20, start_freq=13, end_freq=80).values.flatten().mean(),
        'wt_rtparietal_gamma_mean': crop_wavelet(wt_rtparietal, start_time_millis=-420, end_time_millis=-20, start_freq=30, end_freq=80).values.flatten().mean(),
        
        'mep_category_absolute': epoch['mep_category_absolute'],
        'mep_category_cmap_across_subjects': epoch['mep_category_cmap_across_subjects'],
        'mep_size': epoch['mep_size'],
        'mep_category_percentile': epoch['mep_category_percentile'],
        'mep_category_cmap': epoch['mep_category_cmap']
    }

    dct['wt_all_g2a'] = dct['wt_all_gamma_mean'] / dct['wt_all_alpha_mean']
    dct['wt_all_b2a'] = dct['wt_all_beta_mean'] / dct['wt_all_alpha_mean']
    dct['wt_all_bg2a'] = dct['wt_all_beta_gamma_mean'] / dct['wt_all_alpha_mean']

    dct['wt_ltm1_g2a'] = dct['wt_ltm1_gamma_mean'] / dct['wt_ltm1_alpha_mean']
    dct['wt_ltm1_b2a'] = dct['wt_ltm1_beta_mean'] / dct['wt_ltm1_alpha_mean']
    dct['wt_ltm1_bg2a'] = dct['wt_ltm1_beta_gamma_mean'] / dct['wt_ltm1_alpha_mean']

    dct['wt_rtm1_g2a'] = dct['wt_rtm1_gamma_mean'] / dct['wt_rtm1_alpha_mean']
    dct['wt_rtm1_b2a'] = dct['wt_rtm1_beta_mean'] / dct['wt_rtm1_alpha_mean']
    dct['wt_rtm1_bg2a'] = dct['wt_rtm1_beta_gamma_mean'] / dct['wt_rtm1_alpha_mean']

    dct['wt_central_g2a'] = dct['wt_central_gamma_mean'] / dct['wt_central_alpha_mean']
    dct['wt_central_b2a'] = dct['wt_central_beta_mean'] / dct['wt_central_alpha_mean']
    dct['wt_central_bg2a'] = dct['wt_central_beta_gamma_mean'] / dct['wt_central_alpha_mean']

    dct['wt_ltdlpfc_g2a'] = dct['wt_ltdlpfc_gamma_mean'] / dct['wt_ltdlpfc_alpha_mean']
    dct['wt_ltdlpfc_b2a'] = dct['wt_ltdlpfc_beta_mean'] / dct['wt_ltdlpfc_alpha_mean']
    dct['wt_ltdlpfc_bg2a'] = dct['wt_ltdlpfc_beta_gamma_mean'] / dct['wt_ltdlpfc_alpha_mean']

    dct['wt_rtdlpfc_g2a'] = dct['wt_rtdlpfc_gamma_mean'] / dct['wt_rtdlpfc_alpha_mean']
    dct['wt_rtdlpfc_b2a'] = dct['wt_rtdlpfc_beta_mean'] / dct['wt_rtdlpfc_alpha_mean']
    dct['wt_rtdlpfc_bg2a'] = dct['wt_rtdlpfc_beta_gamma_mean'] / dct['wt_rtdlpfc_alpha_mean']

    dct['wt_occipital_g2a'] = dct['wt_occipital_gamma_mean'] / dct['wt_occipital_alpha_mean']
    dct['wt_occipital_b2a'] = dct['wt_occipital_beta_mean'] / dct['wt_occipital_alpha_mean']
    dct['wt_occipital_bg2a'] = dct['wt_occipital_beta_gamma_mean'] / dct['wt_occipital_alpha_mean']

    dct['wt_ltoccipital_g2a'] = dct['wt_ltoccipital_gamma_mean'] / dct['wt_ltoccipital_alpha_mean']
    dct['wt_ltoccipital_b2a'] = dct['wt_ltoccipital_beta_mean'] / dct['wt_ltoccipital_alpha_mean']
    dct['wt_ltoccipital_bg2a'] = dct['wt_ltoccipital_beta_gamma_mean'] / dct['wt_ltoccipital_alpha_mean']

    dct['wt_rtoccipital_g2a'] = dct['wt_rtoccipital_gamma_mean'] / dct['wt_rtoccipital_alpha_mean']
    dct['wt_rtoccipital_b2a'] = dct['wt_rtoccipital_beta_mean'] / dct['wt_rtoccipital_alpha_mean']
    dct['wt_rtoccipital_bg2a'] = dct['wt_rtoccipital_beta_gamma_mean'] / dct['wt_rtoccipital_alpha_mean']

    dct['wt_ltparietal_g2a'] = dct['wt_ltparietal_gamma_mean'] / dct['wt_ltparietal_alpha_mean']
    dct['wt_ltparietal_b2a'] = dct['wt_ltparietal_beta_mean'] / dct['wt_ltparietal_alpha_mean']
    dct['wt_ltparietal_bg2a'] = dct['wt_ltparietal_beta_gamma_mean'] / dct['wt_ltparietal_alpha_mean']

    dct['wt_rtparietal_g2a'] = dct['wt_rtparietal_gamma_mean'] / dct['wt_rtparietal_alpha_mean']
    dct['wt_rtparietal_b2a'] = dct['wt_rtparietal_beta_mean'] / dct['wt_rtparietal_alpha_mean']
    dct['wt_rtparietal_bg2a'] = dct['wt_rtparietal_beta_gamma_mean'] / dct['wt_rtparietal_alpha_mean']

    return dct

In [11]:
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(get_dict_of_avg_power)(epoch) for _, epoch in tqdm_notebook(df.iterrows(), total=df.shape[0]))

In [12]:
df_powers = pd.DataFrame(results)
df_powers.shape

(1350, 82)

In [13]:
df_powers.head()

,mep_category_absolute,mep_category_cmap,mep_category_cmap_across_subjects,mep_category_percentile,mep_size,wt_all_alpha_mean,wt_all_b2a,wt_all_beta_gamma_mean,wt_all_beta_mean,wt_all_bg2a,...,wt_rtoccipital_bg2a,wt_rtoccipital_g2a,wt_rtoccipital_gamma_mean,wt_rtparietal_alpha_mean,wt_rtparietal_b2a,wt_rtparietal_beta_gamma_mean,wt_rtparietal_beta_mean,wt_rtparietal_bg2a,wt_rtparietal_g2a,wt_rtparietal_gamma_mean
0,2,0.115301,1.0,2,2.555237,0.183244,1.569231,0.223080,0.287553,1.217388,...,0.168841,0.182202,0.084029,0.260655,0.643166,0.096316,0.167644,0.369517,0.507654,0.132323
1,1,0.040348,1.0,1,0.894165,0.323247,0.652767,0.106871,0.211005,0.330615,...,0.216511,0.432306,0.132109,0.325489,0.629243,0.112545,0.204812,0.345772,0.440096,0.143246
2,1,0.027107,1.0,0,0.600739,0.285888,0.750537,0.137226,0.214570,0.480000,...,0.280038,0.267024,0.107180,0.511905,0.453322,0.131395,0.232058,0.256678,0.185362,0.094888
3,1,0.028739,1.0,0,0.636902,0.440394,0.594428,0.196306,0.261782,0.445750,...,0.459326,0.241344,0.082576,0.276456,0.649197,0.094873,0.179475,0.343174,0.424207,0.117275
4,1,0.026983,1.0,0,0.597992,0.285978,0.778613,0.192856,0.222666,0.674373,...,0.374515,0.526343,0.137121,0.276970,0.689622,0.103798,0.191004,0.374762,0.404690,0.112087
